# Pull Forest Inventory Plot Coordinates and Areas

In [2]:
author = ['Marshall Worsham']
date = 2020-12-20

This notebook contains code that cleans and adds missing data to shapefiles that describe the locations of 14 Kueppers 40x40m forest inventory plots in the East River watershed, Colorado, US, as of August 2020. The code operates on three types of shapefiles for each plot:

1. polygon shapefile representing the footprints of all plots
2. polygon shapefiles representing the footprint of a single plot
2. point shapefile representing the four corners of the plot
3. line shapefile representing the four edges of the plot

The output is a set of final properly named shapefiles for storage in the Kueppers et al. East River/RMBL Drive folder.

## 1. Load libraries

In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import re
from matplotlib import pyplot as plt
from os.path import join, getsize
%matplotlib inline

## 2. Import shapefiles

In [88]:
# Define a few directories
os.getcwd()
directory = '/Users/hmworsham/Desktop/RMBL/Projects/Watershed_Spatial_Dataset'
source_dir = os.sep.join([directory, 'Source'])
scratch_dir = os.sep.join([directory, 'Scratch'])
out_dir = os.sep.join([directory, 'Output'])

# Call a directory storing all shapefiles and list contents
sf_dir = os.sep.join([scratch_dir, 'Kueppers_Plot_Bnd_2020_WGS84UTM13N_Renamed'])
os.listdir(sf_dir)[:10]

['SG-SWR1_Bound_lines_WGS84UTM13N.dbf',
 'ER-GT1_PlotCenter_WGS84UTM13N.shp.xml',
 'XX-CAR1_Bound_lines_WGS84UTM13N.sbx',
 'ER-BME2_Bound_poly_WGS84UTM13N.sbn',
 'XX-PLN1_Bound_lines_WGS84UTM13N.dbf',
 'CC-UC2_Bound_poly_WGS84UTM13N.sbx',
 'ER-APL1_Bound_lines_WGS84UTM13N.shp',
 'XX-PLN2_Bound_pts_WGS84UTM13N.cpg',
 'XX-CAR2_Bound_poly_WGS84UTM13N.dbf',
 'XX-PLN2_Bound_poly_WGS84UTM13N.sbx']

In [311]:
# Isolate the polygon sf containing all plot footprints
kplots = [i for i in os.listdir(sf_dir) if ('Kueppers' in i) and (i.endswith('.shp'))]

# Isolate the shapefiles containing single plot footprints
polys = [i for i in os.listdir(sf_dir) if ('poly' in i) and i.endswith('.shp')]
lines = [i for i in os.listdir(sf_dir) if ('line' in i) and i.endswith('.shp')]
cornerpts = [i for i in os.listdir(sf_dir) if ('pts' in i) and i.endswith('.shp')]
centerpts = [i for i in os.listdir(sf_dir) if ('Center' in i) and i.endswith('.shp')]

assert len(polys) == len(lines) == len(cornerpts) == len(centerpts) == 14

In [62]:
# Import kplots as a pandas geodataframe
kplots_gpdf = gpd.read_file(os.sep.join([sf_dir, kplots[0]]))

In [63]:
# Import members of polys, lines, centers, corners as geodataframes and store together in a geometry-type specific list
polys_gpdf_ls = [gpd.read_file(os.sep.join([sf_dir, i])) for i in polys]
lines_gpdf_ls = [gpd.read_file(os.sep.join([sf_dir, i])) for i in lines]
cornerpts_gpdf_ls = [gpd.read_file(os.sep.join([sf_dir, i])) for i in cornerpts]
centerpts_gpdf_ls = [gpd.read_file(os.sep.join([sf_dir, i])) for i in centerpts]

In [64]:
# Verify the geodataframes load as expected
print(kplots_gpdf.head())
print(polys_gpdf_ls[0])
print(lines_gpdf_ls[0])
print(cornerpts_gpdf_ls[0])
print(centerpts_gpdf_ls[0])

   Id    POLY_AREA     AREA_GEO     CENTROID_X    CENTROID_Y      EXT_MIN_X  \
0   0  1221.776739  1221.863563  327968.771206  4.309996e+06  327949.990483   
1   0  1177.413239  1177.591884  337758.617869  4.302957e+06  337736.807200   
2   0  1287.171970  1287.196562  321927.062151  4.297227e+06  321908.283631   
3   0  1235.951437  1235.963259  320848.986798  4.301036e+06  320829.547759   
4   0  1410.672586  1410.729972  324398.723984  4.313021e+06  324373.194049   

      EXT_MIN_Y      EXT_MAX_X     EXT_MAX_Y   PERIMETER   PERIM_GEO  \
0  4.309978e+06  327989.478774  4.310016e+06  140.646309  140.651312   
1  4.302939e+06  337778.177932  4.302973e+06  139.726001  139.736602   
2  4.297210e+06  321947.804717  4.297248e+06  144.320193  144.321566   
3  4.301015e+06  320867.887399  4.301054e+06  141.227379  141.228053   
4  4.312994e+06  324424.847580  4.313048e+06  150.371128  150.374179   

    GEOMCTR_X  GEOMCTR_Y Plot_ID  PNT_COUNT  \
0 -106.984442  38.922037    None        5.0  

In [65]:
# Import external list of plot names, short-form IDs and center point coordinates
coords = pd.read_csv(os.sep.join([source_dir, 'Kueppers_PlotIDs_Coords.csv']))

## 3. Clean 'All Plots' shapefile

In [208]:
# create a copy of kplots for cleaning
kplots_gpdf_clean = kplots_gpdf.copy()

# inner join coords and kplots_gpdf on equal lat values to populate Plot_ID field with short name for reference
# create temporary merge field 'latjoin'
coords['latjoin'] = round(coords['Latitude'], 4)
kplots_gpdf['latjoin'] = round(kplots_gpdf['GEOMCTR_Y'], 4)

# merge as new gpdf
new_df = kplots_gpdf.merge(coords, how = 'inner', on = 'latjoin')
kplots_gpdf_clean['Plot_ID'] = new_df['SFA_ID']
kplots_gpdf_clean = kplots_gpdf_clean[[
    'Plot_ID',
    'AREA_GEO', 
    'PERIM_GEO', 
    'CENTROID_X', 
    'CENTROID_Y',
    'EXT_MIN_X',
    'EXT_MIN_Y',
    'EXT_MAX_X',
    'EXT_MAX_Y',
    'GEOMCTR_X',
    'GEOMCTR_Y',
    'geometry']]

# Round area and perimeter estimates to proper sigfigs
kplots_gpdf_clean.AREA_GEO = round(kplots_gpdf_clean.AREA_GEO,2)
kplots_gpdf_clean.PERIM_GEO = round(kplots_gpdf_clean.PERIM_GEO,2)

kplots_gpdf_clean.columns = [
    'PLOT_ID',
    'AREA_GEO',
    'PERIM_GEO',
    'CENTROID_X',
    'CENTROID_Y',
    'EXT_MIN_X',
    'EXT_MIN_Y',
    'EXT_MAX_X',
    'EXT_MAX_Y',
    'GEOMCTR_X',
    'GEOMCTR_Y',
    'geometry']

In [209]:
# Show the cleaned geodataframe
kplots_gpdf_clean

,PLOT_ID,AREA_GEO,PERIM_GEO,CENTROID_X,CENTROID_Y,EXT_MIN_X,EXT_MIN_Y,EXT_MAX_X,EXT_MAX_Y,GEOMCTR_X,GEOMCTR_Y,geometry
0,SG-SWR1,1221.86,140.65,327968.771206,4.309996e+06,327949.990483,4.309978e+06,327989.478774,4.310016e+06,-106.984442,38.922037,"POLYGON ((327988.594 4310009.047, 327984.998 4..."
1,XX-PLN1,1177.59,139.74,337758.617869,4.302957e+06,337736.807200,4.302939e+06,337778.177932,4.302973e+06,-106.869902,38.860505,"POLYGON ((337778.178 4302938.646, 337739.318 4..."
2,XX-CAR2,1287.20,144.32,321927.062151,4.297227e+06,321908.283631,4.297210e+06,321947.804717,4.297248e+06,-107.050785,38.805835,"POLYGON ((321942.220 4297244.424, 321946.919 4..."
3,CC-UC1,1235.96,141.23,320848.986798,4.301036e+06,320829.547759,4.301015e+06,320867.887399,4.301054e+06,-107.064186,38.839922,"POLYGON ((320863.247 4301015.870, 320828.662 4..."
4,WG-WGM1,1410.73,150.37,324398.723984,4.313021e+06,324373.194049,4.312994e+06,324424.847580,4.313048e+06,-107.026375,38.948571,"POLYGON ((324423.962 4313019.558, 324394.979 4..."
5,ER-APL1,1944.28,176.89,327423.102519,4.316119e+06,327398.007549,4.316097e+06,327447.832708,4.316140e+06,-106.992277,38.977069,"POLYGON ((327446.948 4316140.152, 327443.430 4..."
6,CC-UC2,1538.63,157.11,319223.426529,4.299901e+06,319196.384466,4.299874e+06,319249.271857,4.299929e+06,-107.082607,38.829368,"POLYGON ((319248.385 4299896.601, 319217.419 4..."
7,XX-PLN2,1818.31,171.10,337801.998507,4.302205e+06,337775.829576,4.302177e+06,337826.470554,4.302231e+06,-106.869228,38.853734,"POLYGON ((337825.587 4302220.530, 337818.147 4..."
8,XX-CAR1,1516.91,157.03,324523.481959,4.299187e+06,324496.662428,4.299165e+06,324549.502483,4.299208e+06,-107.021400,38.824003,"POLYGON ((324548.617 4299202.260, 324536.204 4..."
9,SG-NES2,1472.71,153.70,327781.177429,4.310932e+06,327759.284432,4.310911e+06,327804.486267,4.310953e+06,-106.986841,38.930426,"POLYGON ((327803.601 4310948.312, 327795.439 4..."


In [211]:
# Export the cleaned geodataframe as a shapefile
kplots_gpdf_clean.to_file(os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Shapefiles_2020_WGS84UTM13N', 'AllPlots', 'Kueppers_EastRiver_AllPlots_2020_WGS84UTM13N.shp']))

## 4. Clean the polygon shapefiles

In [168]:
# Print the list of shapefiles
polys

['XX-PLN2_Bound_poly_WGS84UTM13N.shp',
 'CC-UC2_Bound_poly_WGS84UTM13N.shp',
 'SG-SWR1_Bound_poly_WGS84UTM13N.shp',
 'ER-BME1_Bound_poly_WGS84UTM13N.shp',
 'XX-CAR2_Bound_poly_WGS84UTM13N.shp',
 'ER-GT1_Bound_poly_WGS84UTM13N.shp',
 'ER-APU1_Bound_poly_WGS84UTM13N.shp',
 'ER-BME2_Bound_poly_WGS84UTM13N.shp',
 'SG-NES2_Bound_poly_WGS84UTM13N.shp',
 'XX-PLN1_Bound_poly_WGS84UTM13N.shp',
 'CC-UC1_Bound_poly_WGS84UTM13N.shp',
 'WG-WGM1_Bound_poly_WGS84UTM13N.shp',
 'XX-CAR1_Bound_poly_WGS84UTM13N.shp',
 'ER-APL1_Bound_poly_WGS84UTM13N.shp']

In [172]:
# Print one of the geodataframes
print(polys_gpdf_ls[0].shape)
polys_gpdf_ls[0]

# Note these should have a single entry with 16 fields

(1, 16)


,Id,POLY_AREA,AREA_GEO,PERIMETER,PERIM_GEO,CENTROID_X,CENTROID_Y,PNT_COUNT,EXT_MIN_X,EXT_MIN_Y,EXT_MAX_X,EXT_MAX_Y,GEOMCTR_X,GEOMCTR_Y,Plot_ID,geometry
0,0,1818.032892,1818.309599,171.090087,171.103097,337801.998507,4.302205e+06,5.0,337775.829576,4.302177e+06,337826.470554,4.302231e+06,-106.869228,38.853734,XX-PLN2,"POLYGON ((337825.587 4302220.530, 337818.147 4..."


In [205]:
polys_gpdf_clean_ls = []
for sf, i in zip(polys_gpdf_ls, polys):

    # Select columns to preserve
    sf_clean = sf[[
    'Plot_ID',
    'AREA_GEO', 
    'PERIM_GEO', 
    'CENTROID_X', 
    'CENTROID_Y',
    'EXT_MIN_X',
    'EXT_MIN_Y',
    'EXT_MAX_X',
    'EXT_MAX_Y',
    'GEOMCTR_X',
    'GEOMCTR_Y',
    'geometry']]

    # Add plot ID
    sf.Plot_ID = i[:7].strip('_')

    # Round area and perimeter estimates to proper sigfigs
    sf_clean.AREA_GEO = round(sf.AREA_GEO,2)
    sf_clean.PERIM_GEO = round(sf.PERIM_GEO,2)

    # Rename columns
    sf_clean.columns = [
        'PLOT_ID',
        'AREA_GEO',
        'PERIM_GEO',
        'CENTROID_X',
        'CENTROID_Y',
        'EXT_MIN_X',
        'EXT_MIN_Y',
        'EXT_MAX_X',
        'EXT_MAX_Y',
        'GEOMCTR_X',
        'GEOMCTR_Y',
        'geometry']
    
    polys_gpdf_clean_ls.append(sf_clean)

0    XX-PLN2
Name: Plot_ID, dtype: object
0    CC-UC2
Name: Plot_ID, dtype: object
0    SG-SWR1
Name: Plot_ID, dtype: object
0    ER-BME1
Name: Plot_ID, dtype: object
0    XX-CAR2
Name: Plot_ID, dtype: object
0    ER-GT1
Name: Plot_ID, dtype: object
0    ER-APU1
Name: Plot_ID, dtype: object
0    ER-BME2
Name: Plot_ID, dtype: object
0    SG-NES2
Name: Plot_ID, dtype: object
0    XX-PLN1
Name: Plot_ID, dtype: object
0    CC-UC1
Name: Plot_ID, dtype: object
0    WG-WGM1
Name: Plot_ID, dtype: object
0    XX-CAR1
Name: Plot_ID, dtype: object
0    ER-APL1
Name: Plot_ID, dtype: object


In [220]:
## Check to make sure they were processed correctly
print([i.shape for i in polys_gpdf_clean_ls])
polys_gpdf_clean_ls[7]

[(1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12), (1, 12)]


,PLOT_ID,AREA_GEO,PERIM_GEO,CENTROID_X,CENTROID_Y,EXT_MIN_X,EXT_MIN_Y,EXT_MAX_X,EXT_MAX_Y,GEOMCTR_X,GEOMCTR_Y,geometry
0,ER-BME2,1483.96,154.34,325233.10933,4.316933e+06,325213.219764,4.316913e+06,325253.159221,4.316953e+06,-107.017749,38.98397,"POLYGON ((325252.274 4316913.555, 325212.833 4..."


In [207]:
# Export each geodataframe to a shapefile with appropriate name
for sfc, sf in zip(polys_gpdf_clean_ls, polys):
    sfc.to_file(os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Shapefiles_2020_WGS84UTM13N', 'Polygons', sf]))

## 5. Clean the lines shapefiles

In [226]:
# Print the list of shapefiles
lines

['ER-APL1_Bound_lines_WGS84UTM13N.shp',
 'XX-CAR1_Bound_lines_WGS84UTM13N.shp',
 'ER-BME1_Bound_lines_WGS84UTM13N.shp',
 'SG-NES2_Bound_lines_WGS84UTM13N.shp',
 'CC-UC2_Bound_lines_WGS84UTM13N.shp',
 'ER-APU1_Bound_lines_WGS84UTM13N.shp',
 'XX-PLN1_Bound_lines_WGS84UTM13N.shp',
 'SG-SWR1_Bound_lines_WGS84UTM13N.shp',
 'XX-PLN2_Bound_lines_WGS84UTM13N.shp',
 'CC-UC1_Bound_lines_WGS84UTM13N.shp',
 'WG-WGM1_Bound_lines_WGS84UTM13N.shp',
 'ER-GT1_Bound_lines_WGS84UTM13N.shp',
 'ER-BME2_Bound_lines_WGS84UTM13N.shp',
 'XX-CAR2_Bound_lines_WGS84UTM13N.shp']

In [229]:
# Print one of the geodataframes
print([i.shape for i in lines_gpdf_ls])
lines_gpdf_ls[0]

# Note these should have a single entry with 3 fields

[(1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3), (1, 3)]


,Id,Plot_ID,geometry
0,0,None,"LINESTRING (327397.123 4316140.235, 327446.948..."


In [230]:
lines_gpdf_clean_ls = []
for sf, name in zip(lines_gpdf_ls, lines):
    
    # Rename and drop columns
    sf_clean = sf.rename(columns = {'Plot_ID':'PLOT_ID'})
    sf_clean = sf_clean.drop(columns = 'Id')
    
    # Add Plot ID
    sf_clean.PLOT_ID = name[:7].strip('_')

    # Export to shapefile 
    sf_clean.to_file(os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Shapefiles_2020_WGS84UTM13N', 'Lines', name]))

    # Append to list
    lines_gpdf_clean_ls.append(sf_clean)

## 6. Clean the center points shapefiles

In [237]:
# Print the list of shapefiles
centerpts

['XX-CAR2_PlotCenter_WGS84UTM13N.shp',
 'XX-PLN2_PlotCenter_WGS84UTM13N.shp',
 'CC-UC2_PlotCenter_WGS84UTM13N.shp',
 'ER-BME1_PlotCenter_WGS84UTM13N.shp',
 'SG-SWR1_PlotCenter_WGS84UTM13N.shp',
 'XX-CAR1_PlotCenter_WGS84UTM13N.shp',
 'ER-APL1_PlotCenter_WGS84UTM13N.shp',
 'ER-BME2_PlotCenter_WGS84UTM13N.shp',
 'ER-APU1_PlotCenter_WGS84UTM13N.shp',
 'ER-GT1_PlotCenter_WGS84UTM13N.shp',
 'WG-WGM1_PlotCenter_WGS84UTM13N.shp',
 'XX-PLN1_PlotCenter_WGS84UTM13N.shp',
 'CC-UC1_PlotCenter_WGS84UTM13N.shp',
 'SG-NES2_PlotCenter_WGS84UTM13N.shp']

In [238]:
# Print one of the geodataframes
print([i.shape for i in centerpts_gpdf_ls])
centerpts_gpdf_ls

# Note these should have a single entry with 5 fields

[(1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5), (1, 5)]


[   Id         XCoord        YCoord  Plot_ID                        geometry
 0   0  321926.176021  4.297228e+06  XX-CAR2  POINT (321926.176 4297228.027),
    Id         XCoord        YCoord  Plot_ID                        geometry
 0   0  337801.114768  4.302205e+06  XX-PLN2  POINT (337801.115 4302205.348),
    Id         XCoord        YCoord Plot_ID                        geometry
 0   0  319222.540102  4.299902e+06  CC-UC2  POINT (319222.540 4299902.141),
    Id         XCoord        YCoord  Plot_ID                        geometry
 0   0  324116.442222  4.316076e+06  ER-BME1  POINT (324116.442 4316076.197),
    Id         XCoord        YCoord Plot_ID                        geometry
 0   0  327967.886322  4.309997e+06  WG-SG2  POINT (327967.886 4309996.908),
    Id         XCoord        YCoord  Plot_ID                        geometry
 0   0  324522.596256  4.299187e+06  XX-CAR1  POINT (324522.596 4299187.326),
    Id         XCoord        YCoord  Plot_ID                        geomet

In [247]:
centerpts_gpdf_clean_ls = []
for sf, name in zip(centerpts_gpdf_ls, centerpts):
    
    # Rename and drop columns
    sf_clean = sf.drop(columns = 'Id')
    sf_clean = sf_clean.rename(columns = {'Plot_ID':'PLOT_ID', 'XCoord':'X_COORD', 'YCoord':'Y_COORD'})
    sf_clean = sf_clean[['PLOT_ID', 'X_COORD', 'Y_COORD', 'geometry']]
    
    # Add Plot ID
    sf_clean.PLOT_ID = name[:7].strip('_')

    # Export to shapefile 
    sf_clean.to_file(os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Shapefiles_2020_WGS84UTM13N', 'Center_Points', name]))

    # Append to list
    centerpts_gpdf_clean_ls.append(sf_clean)

In [248]:
centerpts_gpdf_clean_ls

[   PLOT_ID        X_COORD       Y_COORD                        geometry
 0  XX-CAR2  321926.176021  4.297228e+06  POINT (321926.176 4297228.027),
    PLOT_ID        X_COORD       Y_COORD                        geometry
 0  XX-PLN2  337801.114768  4.302205e+06  POINT (337801.115 4302205.348),
   PLOT_ID        X_COORD       Y_COORD                        geometry
 0  CC-UC2  319222.540102  4.299902e+06  POINT (319222.540 4299902.141),
    PLOT_ID        X_COORD       Y_COORD                        geometry
 0  ER-BME1  324116.442222  4.316076e+06  POINT (324116.442 4316076.197),
    PLOT_ID        X_COORD       Y_COORD                        geometry
 0  SG-SWR1  327967.886322  4.309997e+06  POINT (327967.886 4309996.908),
    PLOT_ID        X_COORD       Y_COORD                        geometry
 0  XX-CAR1  324522.596256  4.299187e+06  POINT (324522.596 4299187.326),
    PLOT_ID        X_COORD       Y_COORD                        geometry
 0  ER-APL1  327422.217745  4.316120e+06  POINT

## 7. Clean the corner points shapefiles

In [233]:
# Print the list of shapefiles
cornerpts

['XX-PLN2_Bound_pts_WGS84UTM13N.shp',
 'ER-BME2_Bound_pts_WGS84UTM13N.shp',
 'ER-APL1_Bound_pts_WGS84UTM13N.shp',
 'XX-CAR2_Bound_pts_WGS84UTM13N.shp',
 'CC-UC1_Bound_pts_WGS84UTM13N.shp',
 'ER-APU1_Bound_pts_WGS84UTM13N.shp',
 'ER-BME1_Bound_pts_WGS84UTM13N.shp',
 'XX-PLN1_Bound_pts_WGS84UTM13N.shp',
 'WG-WGM1_Bound_pts_WGS84UTM13N.shp',
 'CC-UC2_Bound_pts_WGS84UTM13N.shp',
 'SG-NES2_Bound_pts_WGS84UTM13N.shp',
 'ER-GT1_Bound_pts_WGS84UTM13N.shp',
 'SG-SWR1_Bound_pts_WGS84UTM13N.shp',
 'XX-CAR1_Bound_pts_WGS84UTM13N.shp']

In [392]:
# Print one of the geodataframes
print([i.shape for i in cornerpts_gpdf_ls])
cornerpts_gpdf_ls

# Note these are mostly 4x7 dataframes, but XX-CAR1, XX-CAR2, XX-PL2 are longer because we didn't have new corner points to process in 2020

[(4, 24), (4, 7), (4, 7), (4, 24), (4, 7), (4, 7), (4, 7), (4, 7), (4, 7), (4, 7), (4, 7), (4, 7), (4, 7), (4, 24)]


[     Comment  Max_PDOP  Max_HDOP           Corr_Type Rcvr_Type    GPS_Date  \
 0       None      12.3       5.8  Postprocessed Code    Geo 7X  2019-09-16   
 1  NE corner       7.8       4.0  Postprocessed Code    Geo 7X  2019-09-16   
 2  SE corner       2.4       1.3  Postprocessed Code    Geo 7X  2019-09-16   
 3  SW corner       4.1       2.1  Postprocessed Code    Geo 7X  2019-09-16   
 
      GPS_Time Update_Sta Feat_Name          Datafile  ...  GPS_Second  \
 0  01:54:43pm        New  Point_ge  BAGSHAW_PL2A.cor  ...    161700.0   
 1  01:59:22pm        New  Point_ge  BAGSHAW_PL2B.cor  ...    161979.0   
 2  02:02:04pm        New  Point_ge  BAGSHAW_PL2C.cor  ...    162141.0   
 3  02:04:54pm        New  Point_ge  BAGSHAW_PL2D.cor  ...    162311.0   
 
    GNSS_Heigh Vert_Prec  Horz_Prec   Std_Dev      Latitude       Longitude  \
 0    3138.714       2.2        2.0  0.193896  38.853614604  -106.869529354   
 1    3138.684       2.0        1.6  0.310542  38.853964070  -106.8694234

In [271]:
print([i.columns for i in cornerpts_gpdf_ls])

[Index(['Comment', 'Max_PDOP', 'Max_HDOP', 'Corr_Type', 'Rcvr_Type', 'GPS_Date',
       'GPS_Time', 'Update_Sta', 'Feat_Name', 'Datafile', 'Unfilt_Pos',
       'Filt_Pos', 'Data_Dicti', 'GPS_Week', 'GPS_Second', 'GNSS_Heigh',
       'Vert_Prec', 'Horz_Prec', 'Std_Dev', 'Latitude', 'Longitude',
       'Point_ID', 'Plot_ID', 'geometry'],
      dtype='object'), Index(['Id', 'XCoord', 'YCoord', 'Corners', 'Horz_Prec', 'Plot_ID',
       'geometry'],
      dtype='object'), Index(['Id', 'XCoord', 'YCoord', 'Corner', 'Horz_Prec', 'Plot_ID', 'geometry'], dtype='object'), Index(['Comment', 'Max_PDOP', 'Max_HDOP', 'Corr_Type', 'Rcvr_Type', 'GPS_Date',
       'GPS_Time', 'Update_Sta', 'Feat_Name', 'Datafile', 'Unfilt_Pos',
       'Filt_Pos', 'Data_Dicti', 'GPS_Week', 'GPS_Second', 'GNSS_Heigh',
       'Vert_Prec', 'Horz_Prec', 'Std_Dev', 'Latitude', 'Longitude',
       'Point_ID', 'Plot_ID', 'geometry'],
      dtype='object'), Index(['Id', 'XCoord', 'YCoord', 'Corners', 'Horz_Prec', 'Plot_ID',
   

In [307]:
len(cornerpts_gpdf_ls)

14

In [310]:
cornerpts

['ER-BME2_Bound_pts_WGS84UTM13N.shp',
 'ER-APL1_Bound_pts_WGS84UTM13N.shp',
 'XX-CAR2_Bound_pts_WGS84UTM13N.shp',
 'CC-UC1_Bound_pts_WGS84UTM13N.shp',
 'ER-APU1_Bound_pts_WGS84UTM13N.shp',
 'ER-BME1_Bound_pts_WGS84UTM13N.shp',
 'XX-PLN1_Bound_pts_WGS84UTM13N.shp',
 'WG-WGM1_Bound_pts_WGS84UTM13N.shp',
 'CC-UC2_Bound_pts_WGS84UTM13N.shp',
 'SG-NES2_Bound_pts_WGS84UTM13N.shp',
 'ER-GT1_Bound_pts_WGS84UTM13N.shp',
 'SG-SWR1_Bound_pts_WGS84UTM13N.shp',
 'XX-CAR1_Bound_pts_WGS84UTM13N.shp']

### First work on the 4x7 dataframes

In [433]:
# For now we'll separate out the errant gpdfs (XX-CAR1, XX-CAR2, XX-PL2)
cornerpts_gpdf_47_ls = cornerpts_gpdf_ls.copy()
cornerpts_47 = cornerpts.copy()
errants = [0,3,13]
for index in sorted(errants, reverse=True):
    del cornerpts_gpdf_47_ls[index]
    del cornerpts_47[index]

In [434]:
cornerpts_47

['ER-BME2_Bound_pts_WGS84UTM13N.shp',
 'ER-APL1_Bound_pts_WGS84UTM13N.shp',
 'CC-UC1_Bound_pts_WGS84UTM13N.shp',
 'ER-APU1_Bound_pts_WGS84UTM13N.shp',
 'ER-BME1_Bound_pts_WGS84UTM13N.shp',
 'XX-PLN1_Bound_pts_WGS84UTM13N.shp',
 'WG-WGM1_Bound_pts_WGS84UTM13N.shp',
 'CC-UC2_Bound_pts_WGS84UTM13N.shp',
 'SG-NES2_Bound_pts_WGS84UTM13N.shp',
 'ER-GT1_Bound_pts_WGS84UTM13N.shp',
 'SG-SWR1_Bound_pts_WGS84UTM13N.shp']

In [435]:
cornerpts_gpdf_clean_ls = []

for sf, name in zip(cornerpts_gpdf_47_ls, cornerpts_47):
    
    # Rename and drop columns
    sf_clean = sf.drop(columns = 'Id')
    sf_clean = sf_clean.rename(columns = {'Plot_ID':'PLOT_ID', 'XCoord':'X_COORD', 'YCoord':'Y_COORD', 'Corners':'CORNER', 'Corner':'CORNER', 'Horz_Prec':'HORZ_PREC'})
    sf_clean = sf_clean[['PLOT_ID', 'CORNER', 'X_COORD', 'Y_COORD', 'HORZ_PREC', 'geometry']].sort_values(by = 'CORNER', axis = 0)
    
    # Add Plot ID
    sf_clean.PLOT_ID = name[:7].strip('_')

    # Append to list
    cornerpts_gpdf_clean_ls.append(sf_clean)

In [436]:
cornerpts_gpdf_clean_ls[10]

,PLOT_ID,CORNER,X_COORD,Y_COORD,HORZ_PREC,geometry
0,SG-SWR1,1NW,327950.705138,4.310016e+06,0.741722,POINT (327949.820 4310016.623)
1,SG-SWR1,2NE,327989.478774,4.310008e+06,1.215190,POINT (327988.594 4310009.047)
2,SG-SWR1,3SE,327985.883228,4.309978e+06,0.693217,POINT (327984.998 4309978.254)
3,SG-SWR1,4SW,327949.990483,4.309982e+06,1.038837,POINT (327949.106 4309982.655)


### Next work on the errant 4x24 dataframes

In [437]:
cornerpts_gpdf_424_ls = [cornerpts_gpdf_ls[0], cornerpts_gpdf_ls[3], cornerpts_gpdf_ls[13]]
cornerpts_424 = [cornerpts[0], cornerpts[3], cornerpts[13]]

In [438]:
print([i.columns for i in cornerpts_gpdf_424_ls])

[Index(['Comment', 'Max_PDOP', 'Max_HDOP', 'Corr_Type', 'Rcvr_Type', 'GPS_Date',
       'GPS_Time', 'Update_Sta', 'Feat_Name', 'Datafile', 'Unfilt_Pos',
       'Filt_Pos', 'Data_Dicti', 'GPS_Week', 'GPS_Second', 'GNSS_Heigh',
       'Vert_Prec', 'Horz_Prec', 'Std_Dev', 'Latitude', 'Longitude',
       'Point_ID', 'Plot_ID', 'geometry'],
      dtype='object'), Index(['Comment', 'Max_PDOP', 'Max_HDOP', 'Corr_Type', 'Rcvr_Type', 'GPS_Date',
       'GPS_Time', 'Update_Sta', 'Feat_Name', 'Datafile', 'Unfilt_Pos',
       'Filt_Pos', 'Data_Dicti', 'GPS_Week', 'GPS_Second', 'GNSS_Heigh',
       'Vert_Prec', 'Horz_Prec', 'Std_Dev', 'Latitude', 'Longitude',
       'Point_ID', 'Plot_ID', 'geometry'],
      dtype='object'), Index(['Comment', 'Max_PDOP', 'Max_HDOP', 'Corr_Type', 'Rcvr_Type', 'GPS_Date',
       'GPS_Time', 'Update_Sta', 'Feat_Name', 'Datafile', 'Unfilt_Pos',
       'Filt_Pos', 'Data_Dicti', 'GPS_Week', 'GPS_Second', 'GNSS_Heigh',
       'Vert_Prec', 'Horz_Prec', 'Std_Dev', 'Latitude'

In [439]:
cornerpts_424

['XX-PLN2_Bound_pts_WGS84UTM13N.shp',
 'XX-CAR2_Bound_pts_WGS84UTM13N.shp',
 'XX-CAR1_Bound_pts_WGS84UTM13N.shp']

In [440]:
cornerpts_gpdf_424_clean_ls = []
for sf, name in zip(cornerpts_gpdf_424_ls, cornerpts_424):

    # Select and rename columns
    sf_clean = sf[['Plot_ID', 'Comment', 'Longitude', 'Latitude', 'Horz_Prec', 'geometry']]
    sf_clean.columns = ['PLOT_ID', 'CORNER', 'X_COORD', 'Y_COORD', 'HORZ_PREC', 'geometry']
    
    # Populate columns with correct values
    sf_clean['X_COORD'] = sf_clean.geometry.x # Update X_COORD with X in epsg:32613
    sf_clean['Y_COORD'] = sf_clean.geometry.y # Update Y_COORD with Y in epsg:32613
    sf_clean['PLOT_ID'] = name[:7].strip('_')
    
    # Append to full centerpoints list
    cornerpts_gpdf_424_clean_ls.append(sf_clean)

#### Fix the corner identifiers to match convention in 2020 shapefiles

In [441]:
cornerpts_gpdf_424_clean_ls[0]
cornerpts_gpdf_424_clean_ls[0]['CORNER'] = ['1NW', '2NE', '3SE', '4SW']

In [442]:
cornerpts_gpdf_424_clean_ls[1]
cornerpts_gpdf_424_clean_ls[1]['CORNER'] = ['1NW', '2NE', '3SE', '4SW']

In [443]:
cornerpts_gpdf_424_clean_ls[2]
cornerpts_gpdf_424_clean_ls[2]['CORNER'] = ['3SE', '2NE', '1NW', '4SW']

In [444]:
cornerpts_gpdf_clean_ls = cornerpts_gpdf_clean_ls + cornerpts_gpdf_424_clean_ls

### Export all to new shapefiles

In [448]:
finalnames = cornerpts_47 + cornerpts_424
# Export each geodataframe to a shapefile with appropriate name
for sfc, name in zip(cornerpts_gpdf_clean_ls, finalnames):
    sfc.to_file(os.sep.join([out_dir, 'Kueppers_EastRiver_Plot_Shapefiles_2020_WGS84UTM13N', 'Corner_Points', name]))